# Multi-Armed Bandit(MAB)

팔이 여러개인 도둑이라는 뜻인데, 특정 부류에 해당하는 문제들의 카테고리를 지칭하는 용어다. 실제로 Multi-Armed Bandit을 구글링해보면 이렇게 팔이 여러개인 생물(?)의 유명한 이미지가 검색된다.

<img src="images/mab.png" style="width: 300px">

- 이 도둑은 슬롯머신마다 평균적으로 반환하는 보상이 슬롯머신마다 시스템적으로 정해져 있다는 사실을 알고 있다. 그래서 자신의 여러 팔을 이용해서 모든 슬롯머신의 레버를 잡고, 매 step마다 1. 슬롯머신 중 하나를 당겨보고 2. 결과를 확인한 뒤 3. 선택한 슬롯머신에 대한 믿음을 업데이트하는 중이다.
    - '믿음'이라는 표현을 쓴 것은 슬롯머신이 반환하는 결과에 대한 불확실성이 항상 존재하기 때문에 슬롯머신에 대한 어떤 인식도 '확신'이 아니라 '믿음'일 수 밖에 없기 때문이다.
- 그래서, 한 step은 아래와 같이 이루어져 있고, 도둑은 이 step을 반복하며 보상을 가장 많이 반환하는 슬롯머신(즉, 최적의 슬롯머신)을 찾아내고자 한다.
    1. (choose action) 슬롯머신 중 하나를 당긴다.
    1. (generate outcome) 슬롯머신을 당긴 결과를 확인한다.
    1. (update belief) 결과를 통해 보상을 도출하고, 당긴 슬롯머신에 대한 믿음을 이 보상을 통해 업데이트한다.

### Exploration-Exploitation tradeoff

합리적인 도둑이라면 매 step마다 이 두 측면을 고려하면서 슬롯머신을 고를 것이다.
- (Exploration) 어떤 슬롯머신이 최적인지 모르니까 골고루 시도해봐야 한다.
- (Exploitation) 보상을 최대화하려면 최적의 슬롯머신의 레버만 당겨야 한다.

당연히 두 부류의 행동에는 각각 장단점이 존재한다.
- Exploration
    - 장점 : 여러 슬롯머신을 선택하는 과정에서 최적의 슬롯머신을 찾게될 수 있다.
    - 단점 : 최적이 아닌 슬롯머신을 선택하는 것을 피할 수 없고, 따라서 기회비용이 발생할 수 밖에 없다.
- Exploitation
    - 장점 : 선택한 슬롯머신이 최적이면 보상을 극대화한다.
    - 단점 : 선택한 슬롯머신이 최적이 아니면 최적의 슬롯머신도 찾지 못한 채 기회비용만 누적시킨다.

그래서 최종적으로 보상을 최대화하면서 최적의 슬롯머신을 찾기 위해 Exploration과 Exploitation을 합리적으로 병행하는 전략이 필요하고, Thompson sampling은 그 전략 중 하나인 셈이다.

## 예시: A/B test

여러개의 선택지들 중에서 최적의 선택지를 최대한 효과적으로 정하는 문제는 저 도둑만 관심을 갖는 문제는 아니다. 구현되어 있는 여러 개의 시스템 중 최적의 지표를 반환할 것으로 기대되는 시스템을 선택하는 A/B 테스트만 보더라도 MAB 문제에 대응시킬 수 있기 때문이다.
1. (choose action) 한 사용자에게 노출시킬 시스템을 선택한다.
1. (generate outcome) 노출된 시스템에서 사용자가 보인 어떤 반응을 관찰한다.
1. (update belief) 이 반응을 가지고, 선택한 시스템을 사용자에게 노출시킨 행동에 대한 보상을 도출하고 선택한 시스템의 성능에 대한 믿음을 업데이트한다.

이렇듯 MAB 문제를 다양한 상황에 대응시킬 수 있는 만큼, 도둑이나 슬롯머신같은 특수한 단어를 대체할 보편적인 용어를 아래와 같이 정의한다.
- Agent(:=도둑)
    - Environment에 대한 믿음을 바탕으로 action을 선택해 Environment에게 전달하는 주체
    - outcome을 reward에 매핑시키고 이 reward를 사용해 Environment에 대한 믿음을 업데이트하는 주체
        - 참고로 $r:$ `outcome` $\to$ `reward`인 함수 $r$을 reward function이라고 하고, 알려져 있다고 가정한다.
- Environment(:=슬롯머신)
    - Agent의 action을 받아 outcome을 반환하는 주체
- Experiment
    - Agent가 Environment에 대한 믿음을 업데이트하는 여러 번의 step으로 구성된 행동

# Definition: MAB problem

Agent가 Environment로부터의 reward 총합을 최대화하며 Enviornment에 대한 믿음을 매 step마다 실시간으로 업데이트하는 전략에 대한 문제로써, $T$번의 step으로 이루어진 Experiment는 아래와 같이 도식화할 수 있다.

<img src="images/experiment_flow.png" alt="Flow of Experiment" width="700px">

## 예시: Bernoulli Bandit

Environment로부터 관찰할 수 있는 outcome이 두 종류뿐이라 각각을 $\{0,1\}$ 중 하나에 대응시킬 수 있어서 reward를 Bernoulli 분포로 모델링할 수 있는 MAB 문제를 말한다. 